In [14]:
import requests
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict, MessageToJson

# ensure to have a key.txt file with your API key in it
with open('key.txt') as f:
    apiKey = f.read().strip()

# set the headers for the request
headers = {'api_key': apiKey}

In [15]:
# importing static data

# get the static data
staticData = requests.get('https://api.wmata.com/gtfs/rail-gtfs-static.zip', headers=headers)

with open('static.zip', 'wb') as f:
    f.write(staticData.content)


In [8]:

# recieving the response of the API vehicle position data
feed = gtfs_realtime_pb2.FeedMessage()

response  = requests.get('https://api.wmata.com/gtfs/rail-gtfsrt-vehiclepositions.pb', headers=headers)
feed.ParseFromString(response.content)

print(MessageToJson(feed))

{
  "header": {
    "gtfsRealtimeVersion": "2.0",
    "incrementality": "FULL_DATASET",
    "timestamp": "1739205551"
  },
  "entity": [
    {
      "id": "0",
      "isDeleted": false,
      "vehicle": {
        "trip": {
          "tripId": "6573299_20121",
          "startTime": "11:42:00",
          "startDate": "20250210",
          "scheduleRelationship": "SCHEDULED",
          "routeId": "RED",
          "directionId": 0
        },
        "position": {
          "latitude": 39.119957,
          "longitude": -77.164604,
          "bearing": 148.0
        },
        "currentStopSequence": 1,
        "currentStatus": "STOPPED_AT",
        "timestamp": "1739205551",
        "stopId": "PF_A15_C",
        "vehicle": {
          "id": "109",
          "label": "143",
          "licensePlate": "8_7044-7045.7055-7054.7576-7577.7543-7542"
        },
        "occupancyStatus": "EMPTY"
      }
    },
    {
      "id": "1",
      "isDeleted": false,
      "vehicle": {
        "trip": {
    

In [9]:
print(MessageToDict(feed))

{'header': {'gtfsRealtimeVersion': '2.0', 'incrementality': 'FULL_DATASET', 'timestamp': '1739205551'}, 'entity': [{'id': '0', 'isDeleted': False, 'vehicle': {'trip': {'tripId': '6573299_20121', 'startTime': '11:42:00', 'startDate': '20250210', 'scheduleRelationship': 'SCHEDULED', 'routeId': 'RED', 'directionId': 0}, 'position': {'latitude': 39.119957, 'longitude': -77.164604, 'bearing': 148.0}, 'currentStopSequence': 1, 'currentStatus': 'STOPPED_AT', 'timestamp': '1739205551', 'stopId': 'PF_A15_C', 'vehicle': {'id': '109', 'label': '143', 'licensePlate': '8_7044-7045.7055-7054.7576-7577.7543-7542'}, 'occupancyStatus': 'EMPTY'}}, {'id': '1', 'isDeleted': False, 'vehicle': {'trip': {'tripId': '6573318_20121', 'startTime': '11:45:00', 'startDate': '20250210', 'scheduleRelationship': 'SCHEDULED', 'routeId': 'RED', 'directionId': 1}, 'position': {'latitude': 39.061802, 'longitude': -77.0535, 'bearing': 136.0}, 'currentStopSequence': 1, 'currentStatus': 'STOPPED_AT', 'timestamp': '173920555

In [11]:
currentdata = MessageToDict(feed)

print(currentdata['entity'][0])

{'id': '0', 'isDeleted': False, 'vehicle': {'trip': {'tripId': '6573299_20121', 'startTime': '11:42:00', 'startDate': '20250210', 'scheduleRelationship': 'SCHEDULED', 'routeId': 'RED', 'directionId': 0}, 'position': {'latitude': 39.119957, 'longitude': -77.164604, 'bearing': 148.0}, 'currentStopSequence': 1, 'currentStatus': 'STOPPED_AT', 'timestamp': '1739205551', 'stopId': 'PF_A15_C', 'vehicle': {'id': '109', 'label': '143', 'licensePlate': '8_7044-7045.7055-7054.7576-7577.7543-7542'}, 'occupancyStatus': 'EMPTY'}}


In [4]:
# recieving the response of the API trip update data
feed2 = gtfs_realtime_pb2.FeedMessage()
response2  = requests.get('https://api.wmata.com/gtfs/rail-gtfsrt-tripupdates.pb', headers=headers)
feed2.ParseFromString(response2.content)

print(MessageToJson(feed2))

{
  "header": {
    "gtfsRealtimeVersion": "2.0",
    "incrementality": "FULL_DATASET",
    "timestamp": "1739184484"
  },
  "entity": [
    {
      "id": "0",
      "tripUpdate": {
        "trip": {
          "tripId": "6573371_20121",
          "startTime": "05:50:00",
          "startDate": "20250210",
          "scheduleRelationship": "SCHEDULED",
          "routeId": "RED",
          "directionId": 0
        },
        "stopTimeUpdate": [
          {
            "stopSequence": 1,
            "departure": {
              "time": "1739184600",
              "uncertainty": 0
            },
            "stopId": "PF_A15_C",
            "scheduleRelationship": "SCHEDULED"
          },
          {
            "stopSequence": 2,
            "arrival": {
              "time": "1739184790",
              "uncertainty": 0
            },
            "stopId": "PF_A14_C",
            "scheduleRelationship": "SCHEDULED"
          },
          {
            "stopSequence": 3,
            "arri

In [5]:
def getRealTime():
    # !!! ADD STOP ARRIVAL INFORMATION (POPUPS AND DATA FROM TRIPS.JSON)
    # !!! WILL NEED TO FIND STOP_ID IN TRIPS.JSON AND STOP_TIMES.TXT
    # FIND MATCHING STOP_ID / TRIP_ID SEQUENCE ADD NEW ARRIVAL TIME FROM STOP_TIMES.TXT

    def parseDict(pbu):
        # TAKES THE DATA FROM U (THE PB URL) AND TURNS IT INTO A DICTIONARY
        feed = gtfs_realtime_pb2.FeedMessage()
        url = pbu
        response = requests.get(url, headers=headers)
        feed.ParseFromString(response.content)
        feed2 = MessageToDict(feed)
        return feed2

    def getVehicles(pburl):
        def addVehicleInfo(vehicles):
            with open(r'leaflet\routes.json') as data:
                routesJson = json.loads(data.read())
                for vehicle in vehicles['features']:
                    routei = 0

                    # FINDING A ROUTE_ID MATCH BETWEEN VEHICLES AND ROUTESJSON. CYCLES THROUGH ROUTES TO FIND A MATCH
                    # !!! ASSUMES THAT THERE WILL BE A MATCH. FACTOR IN A NO MATCH BY LOOKING FOR END OF ROUTES LIST.
                    while vehicle['data']['routeId'] != routesJson[routei]['route_id']:
                        routei += 1

                    # IF ROUTE_ID MATCH FOUND
                    if vehicle['data']['routeId'] != routesJson[routei]['route_id']:
                        routei = 0

                    # COPY OVER ROUTESJSON ROUTE_SHORT_NAME AND ROUTE_LONG_NAME TO VEHICLES
                    vehicle['data']['route_short_name'] = routesJson[routei]['route_short_name']
                    vehicle['data']['route_long_name'] = routesJson[routei]['route_long_name']
                    vehicle['data']['route_full_name'] = routesJson[routei]['route_short_name'] + " " + routesJson[routei]['route_long_name']

                return vehicles

        def addVehiclePopups(vehicles):
            for vehicle in vehicles['features']:
                vehicle["properties"]["popupContent"] = f"Route: {vehicle['data']['route_short_name']} " \
                                                        f"<br>Route Name: {vehicle['data']['route_long_name']} " \
                                                        f"<br>TripID: {vehicle['data']['tripId']} " \
                                                        f"<br>VehicleID: {vehicle['data']['vehicleId']}"
            return vehicles

        allVehicles = {}
        allVehicles['type'] = {}
        allVehicles['type'] = 'Feature Collection'
        allVehicles['features'] = []

        feed = parseDict(pburl)
        id = 0
        for value in feed['entity']:
            obj = {}

            # LIST OF SECTIONS
            list = ["type", "properties", "geometry", "data"]
            for i in list: # CREATE SECTIONS
                obj[i] = {}
            obj["type"] = "Feature"

            # START OF DATA SECTION
            tripId = value["vehicle"]["trip"]["tripId"]
            uni = obj["data"]
            uni["vehicleId"] = value["vehicle"]["vehicle"]["id"]
            uni["tripId"] = tripId
            uni["routeId"] = value["vehicle"]["trip"]["routeId"]
            uni["coordinates"] = [value["vehicle"]["position"]["longitude"], value["vehicle"]["position"]["latitude"]]

            # START OF GEOMETRY SECTION
            obj["geometry"]["type"] = "Point"
            obj["geometry"]["coordinates"] = uni["coordinates"]


            # START OF PROPERTIES SECTION
            obj["properties"] = {}
            obj["properties"]['id'] = id

            # ADD INDIVIDUAL VEHICLES TO LIST
            id += 1
            allVehicles['features'].append(obj)
        allVehicles = addVehicleInfo(allVehicles)
        allVehicles = addVehiclePopups(allVehicles)
        return allVehicles

    def getTrips(pburl):
        allTrips = {}
        feed = parseDict(pburl)
        for value in feed['entity']:
            tripId = value['tripUpdate']['trip']['tripId']
            allTrips[tripId] = {}
            allTrips[tripId]['tripId'] = tripId
            allTrips[tripId]['routeId'] = value['tripUpdate']['trip']['routeId']
            if 'delay' in value['tripUpdate']['stopTimeUpdate'][0]['departure']:
                allTrips[tripId]['delay'] = value['tripUpdate']['stopTimeUpdate'][0]['departure']['delay']
            if 'delay' in value['tripUpdate']['stopTimeUpdate'][0]['departure']:
                allTrips[tripId]['time'] = value['tripUpdate']['stopTimeUpdate'][0]['departure']['time']
            if 'delay' in value['tripUpdate']['stopTimeUpdate'][0]['departure']:
                allTrips[tripId]['nextStopId'] = value['tripUpdate']['stopTimeUpdate'][0]['stopId']

        return allTrips


    realtime_list = [
        'https://www.metrostlouis.org/RealTimeData/StlRealTimeVehicles.pb',
        'https://www.metrostlouis.org/RealTimeData/StlRealTimeTrips.pb'
    ]

    for item in realtime_list:

        # if looking at vehicles
        if item == 'https://www.metrostlouis.org/RealTimeData/StlRealTimeVehicles.pb':
            print('writing vehicles...')
            vehicles = getVehicles(item)
            saveTempData(vehicles, r'leaflet\vehicles.json')
            pass
        # if looking at the trips file
        elif item == 'https://www.metrostlouis.org/RealTimeData/StlRealTimeTrips.pb':
            print('writing trips...')
            print(item)
            trips = getTrips(item)
            saveTempData(trips, r'leaflet\trips.json')
            pass
        else:
            print(item)
            print('error')
            return